In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
train = datasets.MNIST(root = "/data", train = True, transform=transforms.ToTensor(), download = True)
test = datasets.MNIST(root = "/data", train = False, transform=transforms.ToTensor(), download = True)

train_data = DataLoader(train, batch_size = 100, shuffle = True)
test_data = DataLoader(test, batch_size = 1000, shuffle = False)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

#x, t 입력
#dataloader에서 처리

#함수 정의
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5, device = device), #(1*28*28) -> (16*24*24)
            nn.ReLU(),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5, device = device), #(16*24*24) -> (32*20*20)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(32*20*20) -> (32*10*10)
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5, device = device), #(32*10*10) -> (64*6*6)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2), #(64*6*6) -> (64*3*3)
        )
        self.g = nn.Sequential(
            nn.Linear(64*3*3,100, device = device),
            nn.ReLU(),
            nn.Linear(100,10, device = device),
        )
    def forward(self,x) :
        x = self.f(x)
        x = x.reshape(x.shape[0], -1)
        x = self.g(x)
        return x
F = NN()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(F.parameters(), lr = 0.002)
epoch = 10

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_data :
#순전파
        y = F(x.to(device))
#손실함수
        loss = loss_func(y, t.to(device))
        loss_sum += loss
#역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch {e+1} | loss {loss_sum / len(train_data)}")

KeyboardInterrupt: 

In [ ]:
cnt = 0;

for x, t in test_data :
    y = F(x.to(device))
    for i in range(x.shape[0]) :
        if torch.argmax(y[i]) == t.to(device)[i] :
            cnt += 1

print("correct_posibility : %f" %(cnt / len(test)))
torch.save(F,"MNIST.pt")